In [1]:
import tensorflow as tf
from absl import flags
import scipy.sparse as sp

from pathlib import Path

from PPI.utils import load_edgelist_data, sparse_to_info

In [ ]:
# Set hyper-parameters.
FLAGS = flags.FLAGS


In [2]:
data_path = '../data/yeast.edgelist'
adj = load_edgelist_data(data_path)  # Get adjacency matrix in CSR format.

num_nodes = adj.shape[0]
num_edges = adj.sum()


In [3]:
print(adj.shape)
print(type(adj))

(6526, 6526)
<class 'scipy.sparse.csr.csr_matrix'>


In [4]:
# features_ = sparse_to_info(sp.identity(num_nodes))
# coords_, data_, shape_ = features_
# 
# features = sparse_to_info(adj)
# coords, data, shape = features


num_features = features_nonzero = num_nodes


In [6]:
adj_orig = adj - sp.diags(adj.diagonal())
adj_orig.eliminate_zeros()